## Fantasy AI Assistant

This notebook will get you on your way to a fantasy football championship. Start by importing the Agent class we're going to use, setting the config list, and giving some system instructions. Note that `gpt-4o-mini` seems to work well. This model will be using a lot of tokens, running `gpt-4-turbo` with this notebook will ring up a large bill quickly, even with the token compression implemented below.

In [1]:
from autogen import ConversableAgent
import os 

config_list = [{"model": "gpt-4o-mini", "api_key": os.environ["OPENAI_API_KEY"]}]

assistant = ConversableAgent(
    name="Assistant",
    system_message="You are a helpful AI assistant that is an expert in tool use and in fantasy football. It is long past October of 2023, you need to check the current date to search for current news."
    "Before you recommend adding or dropping any player, be very thorough and check all tools available to you to get the most up to date information possible regarding that player."
    "Return 'TERMINATE' when the task is done.",
    llm_config={"config_list": config_list},
)

Creating the user proxy agent that acts as the stand in for you.

In [2]:
# The user proxy agent is used for interacting with the assistant agent
# and executes tool calls.
user_proxy = ConversableAgent(
    name="User",
    llm_config=False,
    is_termination_msg=lambda msg: msg.get("content") is not None and "TERMINATE" in msg["content"],
    human_input_mode="NEVER",
)


Now we start to create the tools that our assistant can use. There is potential for some change here, as these are written as tools rather than functions. When `autogen` hits their roadmap target of letting docker code execution agents take in user-defined functions, I will refactor this.

In [3]:
from langchain_community.utilities import GoogleSearchAPIWrapper
from langchain_core.tools import Tool
from autogen import register_function

def google_it(query: str) -> str:
    "Search Google for keywords"
    #this is going to assume you have your environment variables set for GOOGLE_API and GOOGLE_CSE
    search = GoogleSearchAPIWrapper()

    google_search = Tool(
        name="google_search",
        description="Search Google for recent results.",
        func=search.run)
    
    return google_search({'query': query})

register_function(
    google_it,
    caller=assistant,  # The assistant agent can suggest calls to the calculator.
    executor=user_proxy,  # The user proxy agent can execute the calculator calls.
    name="google_it",  # By default, the function name is used as the tool name.
    description="Searches google for recent results using the input as keywords",  # A description of the tool.
)


In [4]:
from langchain_community.tools.reddit_search.tool import RedditSearchRun
from langchain_community.utilities.reddit_search import RedditSearchAPIWrapper
from langchain_community.tools.reddit_search.tool import RedditSearchSchema

reddit_client_id = os.environ['REDDIT_CLIENT_ID']
reddit_client_secret = os.environ['REDDIT_CLIENT_SECRET']
reddit_user_agent = "fantasy-assistant"

def check_reddit(player: str, player_team_subreddit: str) -> str:
    "Check Reddit for any recent player news. Inputs are the player and the subreddit for that player's team"

    r_search = RedditSearchRun(
        api_wrapper=RedditSearchAPIWrapper(
            reddit_client_id=reddit_client_id,
            reddit_client_secret=reddit_client_secret,
            reddit_user_agent=reddit_user_agent,
        )
    )

    search_params = RedditSearchSchema(
        query=player, sort="new", time_filter="week", subreddit="fantasyfootball", limit="5"
        )

    result = r_search.run(tool_input=search_params.dict())

    try:
        search_params = RedditSearchSchema(
            query=player, sort="new", time_filter="week", subreddit=player_team_subreddit, limit="5"
            )
        extra_result = r_search.run(tool_input=search_params.dict())
        result += extra_result
    except:
        pass
    return result

register_function(
    check_reddit,
    caller=assistant,  # The assistant agent can suggest calls to the calculator.
    executor=user_proxy,  # The user proxy agent can execute the calculator calls.
    name="check_reddit",  # By default, the function name is used as the tool name.
    description="Check Reddit for any recent player news. Inputs are the player and the subreddit for that player's team",  # A description of the tool.
)

Here you need to make sure you've grabbed your `swid` and `s2` from ESPN's website. It's fairly straightforward when you go to developer tools after logging in. You also need to set up a `config.py` per the comment below the import. It needs to have a variable called `league_team_info` that contains a list of tuples. The values in the tuple must be your nickname for the league, the league ID, and your team's ID within the league. 

In [5]:
import os
from espn_api.football import League
from config import league_team_info
#[('work-friends',1234567,7)]
from espn_ff_toolkit import get_espn_leagues 


swid = os.environ['ESPN_SWID']
espn_s2 = os.environ['ESPN_S2']

year=2023
league_dict = get_espn_leagues(league_team_info,year,espn_s2=espn_s2,swid=swid)

In [6]:
from espn_ff_toolkit import get_rosters, get_free_agents, get_roster_and_projections

def get_rosters_tool()-> str:
    "Get the rosters for all current leagues"
    return get_rosters(league_dict)

register_function(
    get_rosters_tool,
    caller=assistant,  # The assistant agent can suggest calls to the calculator.
    executor=user_proxy,  # The user proxy agent can execute the calculator calls.
    description = "Get the rosters for all current leagues"
)

def get_free_agents_tool() -> str:
    "Get the current free agents in all leagues"
    return get_free_agents(league_dict)

register_function(
    get_free_agents_tool,
    caller=assistant,  # The assistant agent can suggest calls to the calculator.
    executor=user_proxy,  # The user proxy agent can execute the calculator calls.
    description = "Get the current free agents in all leagues"
)

def get_roster_and_projections_tool() -> str:
    "Get the rosters for all current leagues and also all projected and past stats"
    return get_roster_and_projections(league_dict)

register_function(
    get_roster_and_projections_tool,
    caller=assistant,  # The assistant agent can suggest calls to the calculator.
    executor=user_proxy,  # The user proxy agent can execute the calculator calls.
    description = "Get the rosters for all current leagues and also all projected and past stats"
)

In [7]:
from datetime import datetime
def get_current_date() -> str:
    "Get the current date, because it is not October 2023"
    current_date = datetime.now()
    return str(current_date)

register_function(
    get_current_date,
    caller=assistant,  # The assistant agent can suggest calls to the calculator.
    executor=user_proxy,  # The user proxy agent can execute the calculator calls.
    description="Find out what the current date is",  # A description of the tool.
)

In [8]:
from autogen.agentchat.contrib.capabilities.text_compressors import LLMLingua
from autogen.agentchat.contrib.capabilities.transforms import TextMessageCompressor
from autogen.agentchat.contrib.capabilities import transform_messages

llm_lingua = LLMLingua()
text_compressor = TextMessageCompressor(text_compressor=llm_lingua)
context_handling = transform_messages.TransformMessages(transforms=[text_compressor])
context_handling.add_to_agent(assistant)

c:\Users\alexl_g8yj9pc\anaconda3\envs\fantasy-llm\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
chat_result = user_proxy.initiate_chat(assistant, message="Give me any recent news from reddit and google about my rosters")

User (to Assistant):

Give me any recent news from reddit and google about my rosters

--------------------------------------------------------------------------------
7 tokens saved with text compression.

>>>>>>>> USING AUTO REPLY...
Assistant (to User):

***** Suggested tool call (call_YU8hUNvTrU3PXvJlJ7ueXJYV): get_current_date *****
Arguments: 
{}
*********************************************************************************

--------------------------------------------------------------------------------

>>>>>>>> EXECUTING FUNCTION get_current_date...
User (to Assistant):

User (to Assistant):

***** Response from calling tool (call_YU8hUNvTrU3PXvJlJ7ueXJYV) *****
2024-08-30 09:50:16.181391
**********************************************************************

--------------------------------------------------------------------------------
12 tokens saved with text compression.

>>>>>>>> USING AUTO REPLY...
Assistant (to User):

***** Suggested tool call (call_Lib6CGLNZ7m9bl

c:\Users\alexl_g8yj9pc\anaconda3\envs\fantasy-llm\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `GoogleSearchAPIWrapper` was deprecated in LangChain 0.0.33 and will be removed in 0.3.0. An updated version of the class exists in the langchain-google-community package and should be used instead. To use it run `pip install -U langchain-google-community` and import as `from langchain_google_community import GoogleSearchAPIWrapper`.
  warn_deprecated(
c:\Users\alexl_g8yj9pc\anaconda3\envs\fantasy-llm\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The method `BaseTool.__call__` was deprecated in langchain-core 0.1.47 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(



>>>>>>>> EXECUTING FUNCTION check_reddit...

>>>>>>>> EXECUTING FUNCTION get_rosters_tool...
User (to Assistant):

User (to Assistant):

***** Response from calling tool (call_Lib6CGLNZ7m9blUAWV8fTRpK) *****
Jul 10, 2024 ... Breaking down which fantasy football news reports you should trust or ignore. We discuss on the WalterFootball.com After Dark Show for July ... It's the 2024 edition of 10 Lists of 10! nbc_ffhh_wrapproach_240827.jpg. Matthew Berry's 10 Lists of 10 for 2024 Fantasy Football Season. From nbcsports.com. 10. Jul 24, 2024 ... ... NEWS Today August 17, 2024. BARBER HYPE New 1M views · 1:16:27. Go ... Ranking The Top 30 Wide Receivers for Fantasy Football (2024). Sal ... Jul 28, 2024 ... NFL news out of 2024 training camp recapped by Underdog Fantasy's ... We don't get fantasy points in August if you haven't heard. Ja'Lynn ... The Guru's 2024 Fantasy Football Draft Plan · From fantasypoints.com. 23. 5. 78 ... Fantasy football news from former ESPN NFL Insider Adam Caplan

Token indices sequence length is longer than the specified maximum sequence length for this model (2926 > 512). Running this sequence through the model will result in indexing errors


1351 tokens saved with text compression.

>>>>>>>> USING AUTO REPLY...
Assistant (to User):

Here's the summary of recent fantasy football news, insights from Reddit regarding Demarco Murray, and current rosters:

### Recent Fantasy Football News (August 2024)
1. **Matthew Berry's 10 Lists of 10** - An analysis of player rankings and trends for the upcoming season.
2. **Fantasy Draft Plans** - Emphasis on prioritizing wide receivers early in drafts due to depth in running back talent.
3. **Najee Harris as a Sleeper** - A Reddit post highlighted Najee Harris as the main targeted sleeper running back, with insights into offensive line improvements and coaching changes benefiting his potential.
4. **Injuries and Impact** - AJ Dillon's injury and Jaylen Warren's hamstring issues may affect running back dynamics for various teams.
5. **General Observations** - Players like D’Andre Swift and Josh Jacobs are also highlighted as players to consider in drafts, especially if they're available in

In [10]:
from IPython.display import Markdown as md
md(chat_result.summary)

Here's the summary of recent fantasy football news, insights from Reddit regarding Demarco Murray, and current rosters:

### Recent Fantasy Football News (August 2024)
1. **Matthew Berry's 10 Lists of 10** - An analysis of player rankings and trends for the upcoming season.
2. **Fantasy Draft Plans** - Emphasis on prioritizing wide receivers early in drafts due to depth in running back talent.
3. **Najee Harris as a Sleeper** - A Reddit post highlighted Najee Harris as the main targeted sleeper running back, with insights into offensive line improvements and coaching changes benefiting his potential.
4. **Injuries and Impact** - AJ Dillon's injury and Jaylen Warren's hamstring issues may affect running back dynamics for various teams.
5. **General Observations** - Players like D’Andre Swift and Josh Jacobs are also highlighted as players to consider in drafts, especially if they're available in later rounds.

### Insights from Reddit on Demarco Murray
- There was no specific mention of Demarco Murray in the recent Reddit analyses. Instead, the focus was on Najee Harris as a top-target running back sleeper.
- The conversation centers around running backs and players who are pivotal in this year's drafts, such as Jacobs, with historical context around coaching influence.

### Current Rosters Overview
1. **Squad League**: 
   - Ja'Marr Chase, Davante Adams, Josh Allen, Rhamondre Stevenson, Alvin Kamara, Alexander Mattison, George Kittle, Courtland Sutton, Jordan Addison, Tua Tagovailoa, Evan McPherson, Puka Nacua, Jake Ferguson, Isaiah Likely, Chase Brown, Jonathan Mingo, K.J. Osborn.
   
2. **Work League**: 
   - Tyreek Hill, Garrett Wilson, Jalen Hurts, Isiah Pacheco, Raheem Mostert, 49ers D/ST, Michael Pittman Jr., Tyler Allgeier, Kyler Murray, Trey McBride, Kyren Williams, Colts D/ST, Bears D/ST, Darren Waller, Tyler Boyd, Tyler Bass.

### Conclusion
There are no immediate updates on Demarco Murray as he is not primarily involved in the current discussions; the focus is shifting towards other running backs for the fantasy season. 

